In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [16]:
df=pd.read_csv('/kaggle/input/amazon-ratings/ratings_Beauty.csv')

In [17]:
df.head(101)

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200
...,...,...,...,...
96,A1BG8QW55XHN6U,1304651118,5.0,1385424000
97,A6R426V4J7AOM,1304651347,5.0,1385856000
98,A1BG8QW55XHN6U,1304665704,5.0,1385596800
99,AZ4FR14IPKML7,1304665720,5.0,1388966400


In [18]:
df=df.drop('Timestamp',axis=1)
LE=LabelEncoder()
df['UserId']=LE.fit_transform(df['UserId'])
df['ProductId']=LE.fit_transform(df['ProductId'])

In [19]:
df

,UserId,ProductId,Rating
0,725046,0,5.0
1,814606,1,3.0
2,313101,1,5.0
3,291075,2,4.0
4,802842,3,1.0
...,...,...,...
2023065,759765,249269,5.0
2023066,759765,249270,5.0
2023067,1035084,249271,5.0
2023068,1195527,249272,5.0


In [20]:

train,test=train_test_split(df,test_size=0.2)

In [21]:
train

,UserId,ProductId,Rating
187768,1042516,12833,5.0
944349,325864,86488,4.0
1812202,289529,201727,5.0
186882,1062778,12787,5.0
1588741,20553,165587,1.0
...,...,...,...
1215254,1196846,116802,5.0
303893,619051,22547,5.0
923525,271540,84377,5.0
283818,821120,20439,5.0


In [22]:
no_of_rated_products_per_user = df.groupby(by='UserId')['Rating'].count().sort_values(ascending=False)
no_of_rated_products_per_user.head(25)

UserId
821601     389
391642     336
835886     326
597619     278
831533     276
1083909    275
1073877    269
882222     259
196335     249
247870     225
1020770    211
679025     209
1083122    205
428263     186
265249     182
1203797    173
1139330    172
467603     170
274179     168
906185     164
1006031    155
1020175    154
200816     152
127948     151
523349     150
Name: Rating, dtype: int64

In [23]:
nearest_neighbors=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=5)

In [24]:
nearest_neighbors

NearestNeighbors(algorithm='brute', metric='cosine')

In [31]:
nearest_neighbors.fit(train[['UserId','ProductId']])

NearestNeighbors(algorithm='brute', metric='cosine')

recommendation_function

In [32]:
import pandas as pd

def recommendation_system_function(userID, n_recommendation, nearest_neighbors):
    user_profile = df[df['UserId'] == userID].drop('Rating', axis=1)

    # Find similar users using nearest neighbors based on user profiles
    distances, indices = nearest_neighbors.kneighbors(user_profile)
    similar_users = indices.flatten()

    # Retrieve products rated by similar users
    user_recommendations = []
    for user in similar_users:
        products_rated_by_user = df[df['UserId'] == user]
        product_list = products_rated_by_user['ProductId'].unique()
        user_recommendations.extend(product_list)
    unique_recommendations = list(set(user_recommendations))

    # Calculate the number of ratings for each recommended product
    product_ratings_count = df[df['ProductId'].isin(unique_recommendations)]['ProductId'].value_counts()

    # Sort the products based on ratings count in descending order
    sorted_recommendations = product_ratings_count.sort_values(ascending=False)

    # Select the top-rated recommendations
    top_recommendations = sorted_recommendations.index[:n_recommendation]

    # Convert recommended product IDs back to original form
    recommended_product_codes = LE.inverse_transform(top_recommendations)

    # Create a DataFrame to store the recommended products and their data
    recommendation_table = pd.DataFrame({'Product ID': top_recommendations, 'Product Code': recommended_product_codes})
    recommendation_table['Rating Count'] = recommendation_table['Product ID'].map(product_ratings_count)
    recommendation_table.set_index('Product ID', inplace=True)

    # Print the recommendation table
    print("Recommended Products:")
    print(recommendation_table)

    return recommended_product_codes


In [33]:
print(df[df['ProductId']=='B003BMGA6C']['UserId'])

Series([], Name: UserId, dtype: int64)


In [34]:
user_id = 105  # Replace with the desired user ID
num_recommendations = 10  # Replace with the desired number of recommendations

recommended_products = recommendation_system_function(user_id, num_recommendations, nearest_neighbors)


Recommended Products:
           Product Code  Rating Count
Product ID                           
105017       B003V264WW           599
215439       B00CE2JC9W           415
84629        B002TSA916           299
61183        B001G7PPNI           254
198410       B00AHF1GK6           248
45936        B0013QOIGC           242
2122         B0000C0XL8           201
106335       B003XKY57S           137
62656        B001IACZEK           134
39800        B000XJ0PG2           127


,UserId,ProductId,Rating
